# Validación de NBV 

En este notebook implementaremos un sistema para reconstrucción via NBV haciendo uso de la arquitectura autoencoder y MLP

* pedirá la dirección a la carpeta contenedora del objeto
* Creará un link simbolico para acceder a las texturas (verifica si no existe ya) y carpetas para almacenar la información obtenida durante el proceso: 
    - Nubes de puntos
    - RGB
    - Profundidad
* Despliega el objeto para verificar si es el que espera el usuario y debe confirmar si es correcto
* Genera una posición y orientación random (podría extraerla de Hintertoiser), importante conocer el bounding box para no colisionar con el objeto
* Inicia el proceso de reconstrucción
* Captura información y almacena 
* Crea grid de 31x31x31
* Procesamiento de IA
* Condición si la cumple repite o finaliza
* Reporte de métricas

In [2]:
import os
import open3d as o3d
import octomap
import torch
import numpy as np
import pandas as pd
from symlink import symbolic_dir
from utils_o3d_v2 import Get_Pointcloud, Get_RGBD, Get_octree, Get_PointcloudGT, scale_and_translate, Get_voxpoints
from MLP import MLP
from utils import net_position_nbv
from dataset_download import download_collection
from utils_metrics import chamfer_distance, Get_cloud_distance, getCobertura
from utils_save import GuardarDS

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
carpeta_iter = "Voxnet/Itera4/"
direccion = "/mnt/6C24E28478939C77/Saulo/ProyectoPHD/"#Direccion a carpeta contenedora
objeto = "objects"
direccion = direccion + objeto + "/"
metricas = {"ID": [],"id_objeto": [], "iteracion_objeto":[],"pose_inicial":[], "nube_puntos":[], "rejilla":[], "nbv":[], "id_anterior":[], "id_siguiente":[], "chamfer":[], "ganancia_cobertura":[], "cobertura":[]}
umbral = 0.0035
listado = os.listdir(direccion)
print(listado)
I = 0
for l in range (0, len(listado)):
    #carpeta = input("A que carpeta quieres acceder?: ") #object folder
    dir_carpeta = direccion + listado[l] + "/"
    if os.path.lexists( dir_carpeta +"Point_cloud/"+ carpeta_iter) == False:
        #os.mkdir(dir_carpeta +"Point_cloud/Voxnet/")
        #os.mkdir(dir_carpeta +"Octree/Voxnet/")
        #os.mkdir(dir_carpeta + "RGB/Voxnet/" )
        #os.mkdir(dir_carpeta + "Depth/Voxnet/")
        os.mkdir(dir_carpeta +"Point_cloud/"+ carpeta_iter)
        os.mkdir(dir_carpeta +"Octree/"+ carpeta_iter)
        os.mkdir(dir_carpeta + "RGB/" + carpeta_iter)
        os.mkdir(dir_carpeta + "Depth/"+ carpeta_iter)

    img_H = 320
    img_W = 240
    #Cargamos los modelos de predicción de posición
    model= MLP().cuda() 
    path_weights = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/position/weights_entrenamiento_MLP_xavier_normal_2.pth' ## Modificar direccion de pesos
    model.load_state_dict(torch.load(path_weights))
    device = torch.cuda.current_device()

    #Inicializamos el octomap
    resolution = .1 # resolucion del octree
    octree = octomap.OcTree(resolution) # inicializamos el octree

    #Cargamos malla
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True)
    material = o3d.visualization.rendering.MaterialRecord() # Create material
    material.albedo_img = o3d.io.read_image( dir_carpeta + '/meshes/texture.png') # Add texture
    mesh = scale_and_translate(mesh)
    #Obtenemos pointcloud GT
    Get_PointcloudGT(dir_carpeta, mesh, carpeta_iter)

    # Raycasting
    mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(mesh1)
    # render for RGBD images
    render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
    render.scene.add_geometry('mesh', mesh, material)
    #Camera vectors setup
    cent = mesh.get_center()
    up = [0, 1, 0]
    poses = np.load("poses.npy")
    eye_init = poses[116]
    eye = eye_init
    fov = 45
    puntos = Get_voxpoints()

    
    
    #print("Inicia el proceso de reconstrucción ...")
    #while condicion == False:
    for i in range(0,15):    
        # RGBD and pointcloud extraction
        Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, dir_carpeta, i, carpeta_iter, save_acc= True)
        Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, i, carpeta_iter)
        #Occupancy grid
        occupancy_probs =  Get_octree(octree, dir_carpeta, i, carpeta_iter, eye, puntos)
        ## Aqui evaluamos si esta completo el modelo en este punto
        CD = chamfer_distance(dir_carpeta, carpeta_iter)
        condicion, coverage_gain = Get_cloud_distance(dir_carpeta, i, carpeta_iter)
        cov = getCobertura(dir_carpeta, carpeta_iter, i, umbral=umbral)
        #print("Chamfer Distance: {}, Cloud distances: {}, # view: {}".format(CD, Distance, i))
        if condicion == True:
            GuardarDS(metricas,I, i, listado[l], eye_init, eye, octree, occupancy_probs, dir_carpeta, carpeta_iter, CD, coverage_gain, cov)
            break
        ## De no estarlo, se consulta a la NN el NBV 
        else:
            grid = np.reshape(occupancy_probs, (1,1,31,31,31))  
            torch_grid = torch.from_numpy(grid)
            #IA-NBV
            output = net_position_nbv(model, torch_grid, device) 
            eye = output.numpy().reshape(3,).astype("double")
            GuardarDS(metricas,I, i, listado[l], eye_init, eye, octree, occupancy_probs, dir_carpeta, carpeta_iter, CD, coverage_gain,cov) 
        #print("nbv:", eye)
        I += 1
    del octree
    del mesh
    del mesh1
    del material
    del scene
    del render

#print(metricas)   
#print("Volví, tonotos!")
#almacena las métricas de error en archivo NPZ
dataframe = pd.DataFrame(metricas, index=None)
dataframe.to_csv('NBV_Voxnet5.csv',index=False)

['60_CONSTRUCTION_SET', 'CHICKEN_NESTING', 'Cole_Hardware_Hammer_Black', 'Curver_Storage_Bin_Black_Small', 'GARDEN_SWING', 'Playmates_Industrial_CoSplinter_Teenage_Mutant_Ninja_Turtle_Action_Figure', 'RedBlack_Nintendo_3DSXL', 'Threshold_Porcelain_Pitcher_White', 'Weisshai_Great_White_Shark', 'Wishbone_Pencil_Case']
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x2dc42580 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x2dc41d30 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x2dc41d30 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x2dc41d30 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x2dc41d30 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits

## Cobertura por objeto

In [2]:
carpeta_iter = "Voxnet/Itera5/"
direccion = "/mnt/6C24E28478939C77/Saulo/ProyectoPHD/"#Direccion a carpeta contenedora
objeto = "objects"
direccion = direccion + objeto + "/"
listado = os.listdir(direccion)
umbral = 0.0035
cov = []
for l in range (0,len(listado)):
    dir_carpeta = direccion + listado[l] + "/"
    cov.append(getCobertura(dir_carpeta,carpeta_iter,i=1,umbral=umbral))
    print("La cobertura del objeto: {} es: {}%".format(listado[l],cov[l]))
cov_arrays = np.asarray(cov)
print("Tenemos una media de: {}, un máximo de {} y mínimo de {}, con un promedio de 3 nuevas vistas por objeto".format(cov_arrays.mean(), cov_arrays.max(), cov_arrays.min()))

La cobertura del objeto: 60_CONSTRUCTION_SET es: 66.22162207781247%
La cobertura del objeto: CHICKEN_NESTING es: 44.8002127013066%
La cobertura del objeto: Cole_Hardware_Hammer_Black es: 54.19357971929484%
La cobertura del objeto: Curver_Storage_Bin_Black_Small es: 50.48726497846114%
La cobertura del objeto: GARDEN_SWING es: 67.46942615239887%
La cobertura del objeto: Playmates_Industrial_CoSplinter_Teenage_Mutant_Ninja_Turtle_Action_Figure es: 66.18707617236555%
La cobertura del objeto: RedBlack_Nintendo_3DSXL es: 88.68709677419355%
La cobertura del objeto: Threshold_Porcelain_Pitcher_White es: 49.03378551318147%
La cobertura del objeto: Weisshai_Great_White_Shark es: 55.28559935639582%
La cobertura del objeto: Wishbone_Pencil_Case es: 69.52674444199867%
Tenemos una media de: 61.18924078874089, un máximo de 88.68709677419355 y mínimo de 44.8002127013066, con un promedio de 3 nuevas vistas por objeto


# Visualización de los resultados

In [6]:
# ploteo nube acumulada

gt = direccion + listado[1] + "/Point_cloud/" + carpeta_iter + "cloud_acc.pcd"
nube_puntos = o3d.io.read_point_cloud(gt)
o3d.visualization.draw_geometries([nube_puntos])

In [19]:
poses.shape

(162, 3)